In [1]:
import sys
import os
import random
import logging
import time
from tqdm.notebook import tqdm
import pandas as pd

sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from ocp import *
from costs import *
from ocp_utils import *
from env_creator import EnvCreator, generate_sdf_rep
from tensor_decomp import apply_tt
from visualization_utils import plot_traj_projections, plot_traj_and_obs_3d

import pybullet as p
import pybullet_data
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from regression import NN_Regressor, GPy_Regressor
from regression import rbf
import numpy.matlib

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)

#### Load data

In [169]:
exp_name = 'var_start_goal_3000samples'
# exp_name = 'start_goal_fixed_300samples_1'
# exp_name = 'mult_obs_var_start_goal_1000samples_1'
exp_name = 'quad_mult_obs_var_start_goal_1000samples_2'
# exp_name = 'quad_mult_obs_var_start_goal_1000samples_two_obstacles'
# exp_name = 'quad_mult_obs_var_start_goal_1000samples_three_obstacles'
# exp_name = 'quad_mult_obs_var_start_goal_1000samples_five_obstacles'

In [170]:
data = np.load('training_data/data_'+ exp_name +'.npy', allow_pickle=True)[()]

x_inputs = data['x_inputs']
x_outputs = data['x_outputs']
obs_set = data['obstacles']
pca = data['pca']

#### Visualize data in pybullet

In [88]:
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [89]:
idx = np.random.randint(len(data['x_inputs']))
x = x_inputs[idx]
y = x_outputs[idx].reshape(-1,12)
obstacles = obs_set[idx]
x0, x_target = x[-24:-12],x[-12:]
quad_id, obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet_quadcopter(x0, x_target, obstacles)

## Try prediction

#### Load MDN model

In [171]:
import pickle as pkl
import tensorflow_probability as tfp
tf.keras.backend.set_floatx('float64')
tfd = tfp.distributions
tfpl = tfp.layers

#### Setup the model (copy from the learning notebook)

n_comps = 10
D_out = 25
D_in = 624
n_comp_params_size = tfpl.IndependentNormal.params_size(event_shape=(D_out,))

params_size = tfpl.MixtureSameFamily.params_size(num_components=n_comps, component_params_size=n_comp_params_size)

mdn = Sequential([
    Dense(256, activation='relu', input_shape=(D_in,), kernel_regularizer = tf.keras.regularizers.l2(1e-2)),
    Dense(256, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(1e-2)),
    Dense(params_size),
    tfpl.MixtureSameFamily(n_comps, tfpl.IndependentNormal(event_shape=(D_out,)))
])


#### Load the weights

mdn.load_weights('model_data/mdn_'+exp_name)

#### Load NN

In [172]:
nn = tf.keras.models.load_model('model_data/nn_'+exp_name+'.h5')

#### Try prediction

In [173]:
model = mdn
mode = 'mdn'
n_sample = 10

In [9]:
idx = np.random.randint(len(data['x_inputs']))
x = x_inputs[idx]
y = x_outputs[idx].reshape(-1,12)
obstacles = obs_set[idx]
x0, x_target = x[-24:-12],x[-12:]
obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet(x0,x_target, obstacles)

if mode == 'nn':
    y_pred = nn.predict(x[None,:])
    y_traj = pca.inverse_transform(y_pred).reshape(-1,12)
elif mode == 'mdn': 
    y_pred = mdn(x[None,:]).sample(n_sample)
    y_traj = pca.inverse_transform(y_pred).reshape(n_sample, -1, 12)
    y_traj, xs_init, us_init  = get_best_mdn_prediction_ddp2(y_traj, x, obstacles, general_obs=True, obj_id=obj_id, add_zeros_dim = 0)

1.2041165339965458


#### Create ddp

In [10]:
T = 100
prob, lin_sys = setup_ilqr_quadcopter_general(T, x0, x_target, obstacles, obj_id)
ddp = crocoddyl.SolverFDDP(prob)

#### Adapt the prediction using lqt

In [61]:
Dx, Du = 12, 6 #dimensions of x and u
lin_sys = create_double_integrator(Dx, Du, 0.05)
xs_lqt = np.hstack([y_traj[:,:3], y_traj[:,6:9], y_traj[:,3:6], y_traj[:,9:]])
xs_init, us_init = create_lqt_init_quad(lin_sys, xs_lqt, x0, x_target, 100)
xs_init = np.hstack([xs_init[:,:3], xs_init[:,6:9], xs_init[:,3:6], xs_init[:,9:]])
us_init = np.array(ddp.problem.quasiStatic(list(xs_init[:-1])))

In [62]:
ddp.problem.calc(list(xs_init[:,:,None]), list(us_init[:,:,None]))

0.801278441678738

#### Solve DDP

In [63]:
from ocp_utils import *

In [64]:
n_iter = 5
xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=n_iter, th_grad = 1e-6, th_stop = 1e-6, x_reg = 1e-2, u_reg = 1e-8)
cost_nn = ddp.cost
feas_nn = ddp.isFeasible
print('Cost nn: {}'.format( cost_nn))
print('Feasible nn: {}'.format( feas_nn))

# print('Cost std:{}, cost nn: {}'.format(cost_std, cost_nn))
# print('Feasible std:{}, Feasible nn: {}'.format(feas_std, feas_nn))

Cost nn: 1.9654951811976686
Feasible nn: True


#### Standard init

In [144]:
xs_init, us_init = create_standard_init(lin_sys, x0, T)
# xs_init = create_linear_init_quad(x0, x_target, T)
# xs_init = np.hstack([xs_init[:,:3], np.zeros((T+1,3)), xs_init[:,3:], np.zeros((T+1,3))])
us_init = np.array(ddp.problem.quasiStatic(list(xs_init[:-1])))  
#init using waypoint:
# x_waypoint = np.array([0,0,-1,0,0,0])
# xs_init, us_init = create_waypoint_init(x0, x_waypoint, x_target,  T)

#### Solve DDP

In [145]:
n_iter = 5
xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=n_iter, th_grad = 1e-6, th_stop = 1e-6)
cost_std = ddp.cost
feas_std = ddp.isFeasible

print('Cost std:{}, cost nn: {}'.format(cost_std, cost_nn))

print('Feasible std:{}, Feasible nn: {}'.format(feas_std, feas_nn))

Cost std:4.414123463805729, cost nn: 1.6940721977099078
Feasible std:True, Feasible nn: True


#### Compare Initialization vs DDP solution

#### Divide data to train and test data

In [174]:
n_samples = len(x_inputs)
indices = np.arange(n_samples)
x_train, x_test, y_train, y_test, train_idx, test_idx = train_test_split(x_inputs, x_outputs, indices, random_state=3, test_size=0.3)

## Compare warmstarting performance in batches

In [175]:
T = 100
n_iter = 5 #number of ddp iterations
model = mdn
mode = 'mdn'
n_sample = 10
th_grad = 1e-6
th_stop = 1e-6
x_reg = 1e-2

In [ ]:
for n_iter in [10]:#, 10, 50]: 
    print('-------------------\n%d'%n_iter)
    cost_nn_set, cost_std_set, cost_mdn_set = [], [], []
    feas_nn_set, feas_std_set, feas_mdn_set = [], [], []
    for idx in range(len(x_test[:100])): #20,30):#
        if idx in [ 39, 417, 142]:
            continue
        #pick one test case
        print(idx)
        x = x_test[idx]
#         print(x)
        y = y_test[idx].reshape(-1,12)
        full_idx = test_idx[idx]
        obstacles = obs_set[full_idx]
#         print(obstacles)
        x0, x_target = x[-24:-12],x[-12:]
        obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet(x0,x_target, obstacles)

        #create ddp
        prob, lin_sys = setup_ilqr_quadcopter_general(T, x0, x_target, obstacles, obj_id)
        ddp = crocoddyl.SolverFDDP(prob)
        
        #### Standard init
        xs_init, us_init = create_standard_init(lin_sys, x0, T)
        us_init = np.array(ddp.problem.quasiStatic(list(xs_init[:-1]))) 
        xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=n_iter, th_grad = th_grad, th_stop = th_stop)
        cost_std = ddp.cost
        feas_std = ddp.isFeasible
        xs_init_std = xs_init.copy()
        
#         #### Linear init
#         xs_init, us_init = create_linear_init(x0, x_target, T)
#         xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=n_iter, th_grad = th_grad, th_stop = th_stop)
#         cost_std = ddp.cost
#         feas_std = ddp.isFeasible
#         xs_init_std = xs_init.copy()

        #### NN init
        y_pred = nn.predict(x[None,:])
        y_traj = pca.inverse_transform(y_pred).reshape(-1,12)
        Dx, Du = 12, 6 #dimensions of x and u
        lin_sys = create_double_integrator(Dx, Du, 0.05)
        xs_lqt = np.hstack([y_traj[:,:3], y_traj[:,6:9], y_traj[:,3:6], y_traj[:,9:]])
        xs_init, us_init = create_lqt_init_quad(lin_sys, xs_lqt, x0, x_target, 100)
        xs_init = np.hstack([xs_init[:,:3], xs_init[:,6:9], xs_init[:,3:6], xs_init[:,9:]])
        us_init = np.array(ddp.problem.quasiStatic(list(xs_init[:-1])))        
        xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=n_iter, th_grad = th_grad, th_stop = th_stop, x_reg = x_reg, u_reg = x_reg)
        cost_nn = ddp.cost
        feas_nn = ddp.isFeasible
        xs_init_nn = xs_init.copy()

        #### MDN init
        y_pred = mdn(x[None,:]).sample(n_sample)
        y_traj = pca.inverse_transform(y_pred).reshape(n_sample, -1, 12)
        y_traj, xs_init, us_init  = get_best_mdn_prediction_ddp2(y_traj, x, obstacles, general_obs=True, obj_id=obj_id, add_zeros_dim = 0)
        Dx, Du = 12, 6 #dimensions of x and u
        lin_sys = create_double_integrator(Dx, Du, 0.05)
        xs_lqt = np.hstack([y_traj[:,:3], y_traj[:,6:9], y_traj[:,3:6], y_traj[:,9:]])
        xs_init, us_init = create_lqt_init_quad(lin_sys, xs_lqt, x0, x_target, 100)
        xs_init = np.hstack([xs_init[:,:3], xs_init[:,6:9], xs_init[:,3:6], xs_init[:,9:]])
        us_init = np.array(ddp.problem.quasiStatic(list(xs_init[:-1])))        
        xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=n_iter, th_grad = th_grad, th_stop = th_stop, x_reg = x_reg, u_reg = x_reg)
        cost_mdn = ddp.cost
        feas_mdn = ddp.isFeasible
        xs_init_mdn = xs_init.copy()

#         print('Cost std:{}, cost nn: {}'.format(cost_std, cost_nn))
#         print('Feasible std:{}, Feasible nn: {}'.format(feas_std, feas_nn))

        cost_nn_set.append(cost_nn)
        cost_mdn_set.append(cost_mdn)
        cost_std_set.append(cost_std) 
        feas_nn_set.append(feas_nn)
        feas_mdn_set.append(feas_mdn)
        feas_std_set.append(feas_std)
        print('Feasible STD: {}, feasible NN: {}, feasible MDN: {}'.format(np.sum(feas_std_set), np.sum(feas_nn_set), np.sum(feas_mdn_set)))
        print('Cost STD: {:.3f}, Cost NN: {:.3f}, Cost MDN: {:.3f}'.format(np.mean(cost_std_set), np.mean(cost_nn_set), np.mean(cost_mdn_set)))    

-------------------
10
0
1.1538687862863202
Feasible STD: 1, feasible NN: 1, feasible MDN: 1
Cost STD: 1.528, Cost NN: 1.528, Cost MDN: 1.528
1
1.13216393239177
Feasible STD: 2, feasible NN: 2, feasible MDN: 2
Cost STD: 1.724, Cost NN: 1.724, Cost MDN: 1.724
2
1.4048334735559727
Feasible STD: 3, feasible NN: 3, feasible MDN: 3
Cost STD: 2.411, Cost NN: 1.887, Cost MDN: 1.991
3
1.60259012688822
Feasible STD: 4, feasible NN: 3, feasible MDN: 4
Cost STD: 2.352, Cost NN: 1.874, Cost MDN: 2.049
4
7.087321154781113
Feasible STD: 5, feasible NN: 4, feasible MDN: 5
Cost STD: 2.320, Cost NN: 1.828, Cost MDN: 1.955
5
1.6547867426008043
Feasible STD: 6, feasible NN: 5, feasible MDN: 6
Cost STD: 2.369, Cost NN: 1.959, Cost MDN: 2.066
6
1.2622354042161632
Feasible STD: 7, feasible NN: 6, feasible MDN: 7
Cost STD: 2.286, Cost NN: 1.931, Cost MDN: 2.036
7
1.6304811476678496
Feasible STD: 8, feasible NN: 7, feasible MDN: 8
Cost STD: 2.261, Cost NN: 1.954, Cost MDN: 2.044
8
1.0791877755149644
Feasible 

In [ ]:
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [ ]:
cost_std_set = np.array(cost_std_set)[:100]
cost_nn_set = np.array(cost_nn_set)[:100]
cost_mdn_set = np.array(cost_mdn_set)[:100]

In [ ]:
cost_std_set = reject_outliers(cost_std_set, m = 4)
cost_mdn_set = reject_outliers(cost_mdn_set, m = 4)
cost_nn_set = reject_outliers(cost_nn_set, m = 4)

In [ ]:
print('Feasible STD: {}, feasible NN: {}, feasible MDN: {}'.format(np.sum(feas_std_set), np.sum(feas_nn_set), np.sum(feas_mdn_set)))


In [ ]:
print('Cost STD: {:.3f}, Cost NN: {:.3f}, Cost MDN: {:.3f}'.format(np.mean(cost_std_set), np.mean(cost_nn_set), np.mean(cost_mdn_set)))    

In [ ]:
exp_name

In [ ]:
data = dict()
data['feas_std_set'] = feas_std_set
data['feas_nn_set'] = feas_nn_set
data['feas_mdn_set'] = feas_mdn_set
data['cost_std_set'] = cost_std_set
data['cost_nn_set'] = cost_nn_set
data['cost_mdn_set'] = cost_mdn_set
np.save('final_data/'+exp_name+'_warmstart_res_'+str(n_iter), data)